In [1]:
import numpy as np
import pandas as pd
from FTS import fts_util
from PyEMD import EMD
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
cores = sns.color_palette(n_colors=21)
warnings.filterwarnings('ignore')

## Grid Search in TAIEX Time Series

In [2]:
# read tsfcm_labels
ts = pd.read_csv('data/TAIEX_1992.csv', index_col=[0], parse_dates=[0])
col = ts.keys().to_list()[0]

file_results = 'out/QE_GS_TAIEX_1992.csv'

# apply EMD to get IMFs
emd = EMD()
imfs = emd.emd(ts[col].values)

ts[col].values.shape

limite = ts[col].values.shape[0]

In [3]:
#fts_util.plot_imfs(ala[col].values, imfs, fs=(20,15))

''' Select MFIs without trending and noise behavior '''

df_fts = pd.DataFrame(ts[col].copy())
df_fts[col] = ts

col_imf = '2+3+residue'
df_fts[col_imf] = imfs[2]+imfs[3]+imfs[4]

In [4]:
from FTS.Models.MV_Lee import STFMV_Convencional_Lee
model = STFMV_Convencional_Lee()

In [ ]:
for k in range(2, limite):
    # run model
    model.fit(X=df_fts[[col, col_imf]].values, k=k)
    mape, mae, mse, rmse, dtw = model.predict(plot=False, SM=False)
    
    # verify and save 
    experimento = {'IMF':col_imf, 'amount':k, 'MAPE':mape, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'DTW':dtw}
    fts_util.update_values_exp(experimento, key_words=['IMF','amount', 'MSE'], fp=file_results, log=False)

In [ ]:
df_view = pd.read_csv(file_results, index_col=['amount'])
df_view

In [ ]:
best = fts_util.best_GS(df_view, 'MSE')

plt.figure(figsize=(10,4))
plt.plot(best.index.values[0], best['MSE'].values[0], '-mo', markersize=15)
plt.title('Best Amount: {} and MSE: {}'.format(best.index.values[0], best['MSE'].values[0]))
plt.plot(df_view['MSE'], '-')
# for i in range(df_view.shape[0]):
#     plt.annotate(df_view.index[i], (df_view.index[i], df_view['MSE'].iloc[i]))
plt.show()